# Volatility Smile Calibration with Automatic Differentiation

This notebook demonstrates calibrating the **SABR stochastic volatility model** to market-implied volatilities using **automatic differentiation** for gradient-based optimization.

## What You'll See
1. Generate synthetic "market" data from known SABR parameters
2. Calibrate SABR model using AD-powered gradient descent
3. Visualize the fitted smile vs market quotes
4. Compare SABR vs Heston calibration

In [ ]:
using Pkg
Pkg.activate("..")

using Quasar
using Plots
gr()  # Use GR backend for plotting

## 1. Generate Synthetic Market Data

We'll create "market" implied volatilities from known SABR parameters, then try to recover them through calibration.

In [ ]:
# "True" market parameters (what we'll try to recover)
true_params = SABRParams(0.25, 1.0, -0.4, 0.5)

# Market setup
F = 100.0   # Forward price
T = 1.0     # 1 year expiry
r = 0.05    # Risk-free rate

# Strike range (70% to 130% moneyness)
strikes = collect(70.0:5.0:130.0)

# Generate "market" implied vols
market_vols = [sabr_implied_vol(F, K, T, true_params) for K in strikes]

println("True SABR Parameters:")
println("  α (alpha) = $(true_params.alpha)")
println("  β (beta)  = $(true_params.beta)")
println("  ρ (rho)   = $(true_params.rho)")
println("  ν (nu)    = $(true_params.nu)")

In [ ]:
# Plot the "market" smile
plot(strikes, market_vols .* 100,
     label="Market Implied Vol",
     xlabel="Strike",
     ylabel="Implied Volatility (%)",
     title="Volatility Smile (Synthetic Market Data)",
     linewidth=2,
     marker=:circle,
     legend=:topright)
vline!([F], label="ATM Forward", linestyle=:dash, color=:gray)

## 2. Calibrate SABR Model

Now we'll calibrate the SABR model to this market data using `calibrate_sabr()`, which uses **automatic differentiation** to compute gradients for optimization.

In [ ]:
# Create market data structure
quotes = [OptionQuote(K, T, 0.0, :call, vol) for (K, vol) in zip(strikes, market_vols)]
smile = SmileData(T, F, r, quotes)

# Calibrate (beta=1.0 is fixed, matching our "market")
@time result = calibrate_sabr(smile; beta=1.0, max_iter=3000, lr=0.01)

println("\nCalibration Result:")
println("  Converged: $(result.converged)")
println("  Iterations: $(result.iterations)")
println("  RMSE: $(round(result.rmse * 10000, digits=2)) bps")
println("\nFitted Parameters:")
println("  α = $(round(result.params.alpha, digits=4)) (true: $(true_params.alpha))")
println("  ρ = $(round(result.params.rho, digits=4)) (true: $(true_params.rho))")
println("  ν = $(round(result.params.nu, digits=4)) (true: $(true_params.nu))")

In [ ]:
# Compute fitted vols
fitted_vols = [sabr_implied_vol(F, K, T, result.params) for K in strikes]

# Plot comparison
plot(strikes, market_vols .* 100,
     label="Market",
     xlabel="Strike",
     ylabel="Implied Volatility (%)",
     title="SABR Calibration: Fitted vs Market",
     linewidth=2,
     marker=:circle,
     markersize=6)

plot!(strikes, fitted_vols .* 100,
      label="SABR Fit",
      linewidth=2,
      linestyle=:dash,
      marker=:square,
      markersize=4)

vline!([F], label="ATM", linestyle=:dot, color=:gray)

In [ ]:
# Plot calibration errors
errors_bps = (fitted_vols .- market_vols) .* 10000

bar(strikes, errors_bps,
    label="Fitting Error",
    xlabel="Strike",
    ylabel="Error (bps)",
    title="Calibration Errors",
    color=:steelblue,
    alpha=0.7)
hline!([0], label="", color=:black, linewidth=1)

## 3. Understanding the SABR Smile

Let's explore how each SABR parameter affects the volatility smile.

In [ ]:
# Effect of rho (skew)
rhos = [-0.6, -0.3, 0.0, 0.3, 0.6]

p = plot(title="Effect of ρ (Correlation) on Smile",
         xlabel="Strike", ylabel="Implied Vol (%)")

for ρ in rhos
    params = SABRParams(0.25, 1.0, ρ, 0.4)
    vols = [sabr_implied_vol(F, K, T, params) * 100 for K in strikes]
    plot!(p, strikes, vols, label="ρ = $ρ", linewidth=2)
end

vline!(p, [F], label="ATM", linestyle=:dash, color=:gray)
p

In [ ]:
# Effect of nu (vol of vol / smile curvature)
nus = [0.1, 0.3, 0.5, 0.7, 0.9]

p = plot(title="Effect of ν (Vol-of-Vol) on Smile",
         xlabel="Strike", ylabel="Implied Vol (%)")

for ν in nus
    params = SABRParams(0.25, 1.0, -0.3, ν)
    vols = [sabr_implied_vol(F, K, T, params) * 100 for K in strikes]
    plot!(p, strikes, vols, label="ν = $ν", linewidth=2)
end

vline!(p, [F], label="ATM", linestyle=:dash, color=:gray)
p

## 4. Compare with Heston Model

Let's also price options using the Heston stochastic volatility model and compare.

In [ ]:
# Heston parameters (roughly matching our SABR smile)
heston = HestonParams(0.0625, 0.0625, 2.0, 0.4, -0.5)  # v0=σ²=0.25², θ=v0, κ=2, σ=0.4, ρ=-0.5

# Price calls with Heston and back out implied vols
S = F * exp(-r * T)  # Spot from forward

heston_prices = [heston_price(S, K, T, r, heston, :call) for K in strikes]

# Compare SABR prices
sabr_prices = [sabr_price(F, K, T, r, result.params, :call) for K in strikes]

plot(strikes, sabr_prices,
     label="SABR",
     xlabel="Strike",
     ylabel="Call Price",
     title="Option Prices: SABR vs Heston",
     linewidth=2)

plot!(strikes, heston_prices,
      label="Heston",
      linewidth=2,
      linestyle=:dash)

## 5. AD Backend Comparison

Quasar supports multiple AD backends. Let's see them in action.

In [ ]:
# Define a simple loss function
function sabr_loss(params_vec)
    α, ρ_raw, ν_raw = params_vec
    α = abs(α)
    ρ = tanh(ρ_raw)
    ν = exp(ν_raw)
    
    sabr = SABRParams(α, 1.0, ρ, ν)
    
    total = 0.0
    for (K, market_vol) in zip(strikes, market_vols)
        model_vol = sabr_implied_vol(F, K, T, sabr)
        total += (model_vol - market_vol)^2
    end
    return total
end

# Test point
x0 = [0.25, 0.0, log(0.3)]

# ForwardDiff backend
println("ForwardDiff gradient:")
@time g_fd = gradient(sabr_loss, x0; backend=ForwardDiffBackend())
println("  ", round.(g_fd, digits=6))

# PureJulia (finite differences) backend
println("\nPureJulia (finite diff) gradient:")
@time g_pj = gradient(sabr_loss, x0; backend=PureJuliaBackend())
println("  ", round.(g_pj, digits=6))

println("\nDifference: ", round.(g_fd .- g_pj, digits=8))

## Summary

This notebook demonstrated:

1. **SABR Model Calibration** - Fitting the industry-standard SABR model to market smiles
2. **Automatic Differentiation** - Using AD for efficient gradient computation in optimization
3. **Parameter Interpretation** - How ρ controls skew and ν controls curvature
4. **Multi-Backend AD** - Quasar's flexible AD system (ForwardDiff, finite differences, with Enzyme/Reactant stubs for GPU)

### Key Takeaways for Interviews
- SABR is the go-to model for equity/FX volatility smiles
- Calibration is an inverse problem solved via optimization
- AD is essential for efficient gradient-based optimization
- Parameter identifiability can be an issue (different params can produce similar smiles)